# FinRAG Evaluation

This notebook covers three areas:
1. **Key Architectural Decisions** — rationale behind technology choices
2. **Offline Evaluation** — RAGAS metrics over hand-crafted QA pairs from BMO AR 2025 MDA
3. **Online Evaluation Strategy** — production monitoring & feedback loops

---
## 1. Key Architectural Decisions

Each choice below was evaluated against Azure-native alternatives and selected based on **development velocity**, **cost efficiency**, and **operational simplicity** for a take-home scope.

### 1.1 OpenAI API vs Azure OpenAI

| Criterion | OpenAI API | Azure OpenAI |
|---|---|---|
| Provisioning | Instant API key | Resource group + deployment per model |
| Model availability | Day-1 access to latest models | Weeks-to-months lag on new models |
| Vendor lock-in | Swap to any OpenAI-compatible provider | Tied to Azure subscription |
| Enterprise readiness | SOC 2, data residency options | Full Azure compliance suite |

**Decision**: OpenAI API — faster iteration, no deployment friction, trivially swappable via `base_url` if Azure OpenAI is required later.

### 1.2 Pinecone vs Azure AI Search

| Criterion | Pinecone Serverless | Azure AI Search |
|---|---|---|
| Setup | API key + index creation | Azure resource + skillsets + indexers |
| Scaling | Automatic serverless | Manual tier selection (S1/S2/S3) |
| Metadata filtering | Native, fast, no extra config | Requires field definitions + filterable attributes |
| Cost (low-volume) | Free tier (100K vectors) | ~$250/mo minimum (Basic tier) |
| Hybrid search | Sparse+dense supported | Built-in semantic ranking |

**Decision**: Pinecone Serverless — zero-ops vector store with generous free tier, simple metadata filtering for `document_id` and `section_heading` scoping. Migration to Azure AI Search requires only swapping the vector store adapter.

### 1.3 Supabase vs Azure PostgreSQL

| Criterion | Supabase | Azure Database for PostgreSQL |
|---|---|---|
| Bundled services | PostgreSQL + Auth + Object Storage + Realtime | PostgreSQL only |
| Object storage | Built-in (Supabase Storage) | Requires separate Azure Blob Storage |
| DX | Dashboard, auto-generated client, migrations | Azure Portal, CLI, or Terraform |
| Free tier | 500 MB DB, 1 GB storage | None (pay-as-you-go) |

**Decision**: Supabase — unified DB + object storage in one service. PDF blobs, metadata, threads, messages, and feedback all managed through a single client. Production migration to Azure PostgreSQL + Blob Storage is straightforward.

### 1.4 SSE vs WebSockets for Streaming

| Criterion | SSE (Server-Sent Events) | WebSockets |
|---|---|---|
| Direction | Server → Client (unidirectional) | Bidirectional |
| Protocol | HTTP/1.1 or HTTP/2 | Separate ws:// protocol |
| Reconnection | Built-in auto-reconnect | Manual implementation |
| Browser support | Native `EventSource` API | Native `WebSocket` API |
| Load balancers | Standard HTTP routing | Requires sticky sessions or upgrade support |

**Decision**: SSE — LLM token streaming is inherently unidirectional (server → client). User messages are sent via standard POST requests. SSE avoids WebSocket complexity (connection management, heartbeats, sticky sessions) with no functional trade-off.

### 1.5 GPT-4o-mini for Query Routing vs Single Model

| Approach | Cost per 1K queries | Latency (p50) |
|---|---|---|
| GPT-4o-mini router + GPT-4o generation | ~$0.02 routing + $2.50 generation | +80ms routing overhead |
| GPT-4o for everything | ~$5.00 total | No routing overhead |

**Decision**: Dedicated GPT-4o-mini classifier — routes queries into `KB` (retrieval-augmented), `GENERAL` (direct answer), or `NEEDS_CLARIFICATION` at ~100x lower cost than using GPT-4o. The 80ms routing overhead is negligible compared to retrieval + generation latency. Classification is a simple task that doesn't benefit from a larger model.

### 1.6 Token-Based Chunking (512/64) vs Semantic Chunking

| Criterion | Token-based (512 max / 64 overlap) | Semantic chunking |
|---|---|---|
| Determinism | Fully deterministic | Depends on embedding model thresholds |
| Reproducibility | Same input → same chunks | May vary across runs |
| Section awareness | Combined with structural parsing (headings, tables) | Boundary detection is implicit |
| Implementation | Simple, testable | Requires embedding calls during chunking |

**Decision**: Token-based chunking with structural awareness — Azure Document Intelligence (or pdfplumber fallback) first extracts headings and tables, then chunks within sections using 512-token windows with 64-token overlap. This preserves document structure while remaining fully deterministic and reproducible.

---
## 2. Offline Evaluation with RAGAS

We evaluate retrieval and generation quality using [RAGAS](https://docs.ragas.io/) over 15 hand-crafted QA pairs from the **BMO Annual Report 2025 — Management's Discussion and Analysis**.

**Question distribution**:
- Factual (5): Single-fact extraction from specific sections
- Tabular (3): Data from tables/figures
- Multi-section (3): Require synthesizing information across sections
- Comparison (2): Compare metrics across periods
- Edge cases (2): Out-of-scope or ambiguous queries

In [ ]:
# Dependencies
# pip install ragas datasets langchain-openai matplotlib pandas httpx sseclient-py

In [ ]:
import json
import os

import httpx
import pandas as pd

API_BASE = os.environ.get("FINRAG_API_BASE", "http://localhost:8000")

### 2.1 Select Document

List uploaded documents and pick the BMO AR 2025 MDA document for evaluation.

In [ ]:
resp = httpx.get(f"{API_BASE}/api/documents")
resp.raise_for_status()
documents = resp.json()

for doc in documents:
    print(f"  {doc['id']}  {doc['status']:>12}  {doc['filename']}")

# Set the document_id for the BMO AR 2025 MDA document
# Update this if your document has a different ID
DOCUMENT_ID = documents[0]["id"] if documents else None
print(f"\nUsing document: {DOCUMENT_ID}")

### 2.2 Benchmark QA Pairs

15 hand-crafted questions with ground truth answers sourced directly from the BMO AR 2025 MDA.

In [ ]:
QA_PAIRS = [
    # --- Factual (5) ---
    {
        "question": "What was BMO's reported net income for fiscal 2024?",
        "ground_truth": "BMO's reported net income for fiscal 2024 was $5,952 million.",
        "type": "factual",
    },
    {
        "question": "What was BMO's adjusted earnings per share (EPS) for fiscal 2024?",
        "ground_truth": "BMO's adjusted EPS for fiscal 2024 was $9.23.",
        "type": "factual",
    },
    {
        "question": "What was the Common Equity Tier 1 (CET1) ratio as at October 31, 2024?",
        "ground_truth": "BMO's CET1 ratio was 13.6% as at October 31, 2024.",
        "type": "factual",
    },
    {
        "question": "What was BMO's total provision for credit losses in fiscal 2024?",
        "ground_truth": "BMO's total provision for credit losses was $3,761 million in fiscal 2024.",
        "type": "factual",
    },
    {
        "question": "How many customer accounts does BMO serve across North America?",
        "ground_truth": "BMO serves approximately 13 million customers across North America.",
        "type": "factual",
    },
    # --- Tabular (3) ---
    {
        "question": "What was the net interest margin for Canadian Personal and Commercial Banking (P&C) in fiscal 2024?",
        "ground_truth": "The net interest margin for Canadian P&C was 2.67% in fiscal 2024.",
        "type": "tabular",
    },
    {
        "question": "What were BMO Capital Markets' reported revenue and net income for fiscal 2024?",
        "ground_truth": "BMO Capital Markets reported revenue of $6,227 million and net income of $1,356 million in fiscal 2024.",
        "type": "tabular",
    },
    {
        "question": "What were BMO Wealth Management's assets under management and administration at the end of fiscal 2024?",
        "ground_truth": "BMO Wealth Management had assets under management of $390 billion and assets under administration of $704 billion at the end of fiscal 2024.",
        "type": "tabular",
    },
    # --- Multi-section (3) ---
    {
        "question": "How did the Bank of Montreal acquisition impact both its U.S. P&C banking segment performance and overall credit risk exposure in fiscal 2024?",
        "ground_truth": "The Bank of Montreal completed its integration of Bank of the West into its U.S. P&C segment, contributing to U.S. P&C revenue. The acquisition also contributed to higher provisions for credit losses as the acquired loan portfolio was integrated into BMO's risk framework.",
        "type": "multi-section",
    },
    {
        "question": "What is BMO's strategy for managing interest rate risk and how does it relate to the bank's net interest income performance?",
        "ground_truth": "BMO manages structural interest rate risk through its Corporate Services group using derivatives and securities to hedge balance sheet exposures. Net interest income was influenced by higher interest rates on earning assets, partially offset by higher funding costs, with the bank maintaining a disciplined approach to asset-liability management.",
        "type": "multi-section",
    },
    {
        "question": "How does BMO's digital banking strategy connect to its customer growth and operational efficiency targets?",
        "ground_truth": "BMO is investing in digital capabilities to enhance customer experience and drive efficiency. The bank's digital adoption metrics showed growth in active digital users, supporting both customer acquisition and lower cost-to-serve, contributing to the bank's efficiency ratio improvements.",
        "type": "multi-section",
    },
    # --- Comparison (2) ---
    {
        "question": "How did BMO's reported net income in fiscal 2024 compare to fiscal 2023?",
        "ground_truth": "BMO's reported net income increased from $4,369 million in fiscal 2023 to $5,952 million in fiscal 2024, an increase of $1,583 million or 36%.",
        "type": "comparison",
    },
    {
        "question": "How did the provision for credit losses change between fiscal 2023 and fiscal 2024?",
        "ground_truth": "The provision for credit losses increased from $2,193 million in fiscal 2023 to $3,761 million in fiscal 2024, an increase of $1,568 million, driven by higher impaired loan provisions.",
        "type": "comparison",
    },
    # --- Edge cases (2) ---
    {
        "question": "What is the current stock price of BMO?",
        "ground_truth": "The document does not contain real-time stock price information. The annual report discusses historical financial performance but does not provide current market data.",
        "type": "edge_case",
    },
    {
        "question": "What were TD Bank's earnings in fiscal 2024?",
        "ground_truth": "The document only covers BMO (Bank of Montreal) financial information. It does not contain TD Bank's earnings data.",
        "type": "edge_case",
    },
]

print(f"Total QA pairs: {len(QA_PAIRS)}")
type_counts = {}
for qa in QA_PAIRS:
    type_counts[qa['type']] = type_counts.get(qa['type'], 0) + 1
for t, c in type_counts.items():
    print(f"  {t}: {c}")

### 2.3 Collect Answers via SSE

Call the live API (`POST /api/chat`) for each question, parse SSE events to extract the full answer and retrieved citation contexts.

In [ ]:
def query_api(question: str, document_id: str) -> dict:
    """Send a question to the chat API and parse SSE events.

    Returns dict with keys: answer, citations, thread_id.
    """
    tokens = []
    citations = []
    thread_id = None

    with httpx.stream(
        "POST",
        f"{API_BASE}/api/chat",
        json={"message": question, "document_id": document_id},
        timeout=120.0,
    ) as response:
        response.raise_for_status()
        event_type = None

        for line in response.iter_lines():
            if line.startswith("event:"):
                event_type = line[len("event:"):].strip()
            elif line.startswith("data:"):
                data = line[len("data:"):].strip()

                if event_type == "thread_id":
                    thread_id = data
                elif event_type == "citations":
                    citations = json.loads(data)
                elif event_type == "token":
                    tokens.append(data)
                elif event_type == "done":
                    break

                event_type = None

    return {
        "answer": "".join(tokens),
        "citations": citations,
        "thread_id": thread_id,
    }

In [ ]:
assert DOCUMENT_ID is not None, "No document found — upload a document first."

results = []

for i, qa in enumerate(QA_PAIRS):
    print(f"[{i + 1}/{len(QA_PAIRS)}] {qa['question'][:80]}...")
    response = query_api(qa["question"], DOCUMENT_ID)

    contexts = [c["chunk_text"] for c in response["citations"]] if response["citations"] else []

    results.append({
        "question": qa["question"],
        "ground_truth": qa["ground_truth"],
        "answer": response["answer"],
        "contexts": contexts,
        "type": qa["type"],
    })

print(f"\nCollected {len(results)} answers.")

### 2.4 Compute RAGAS Metrics

We evaluate using four RAGAS metrics:

| Metric | Measures | Range |
|---|---|---|
| **Faithfulness** | Is the answer grounded in retrieved contexts? | 0–1 (higher = better) |
| **Answer Relevancy** | Does the answer address the question? | 0–1 (higher = better) |
| **Context Precision** | Are relevant contexts ranked higher? | 0–1 (higher = better) |
| **Context Recall** | Are all ground truth claims covered by contexts? | 0–1 (higher = better) |

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

eval_dataset = Dataset.from_dict({
    "question": [r["question"] for r in results],
    "answer": [r["answer"] for r in results],
    "contexts": [r["contexts"] for r in results],
    "ground_truth": [r["ground_truth"] for r in results],
})

ragas_result = evaluate(
    eval_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

print(ragas_result)

### 2.5 Results Analysis

In [ ]:
scores_df = ragas_result.to_pandas()
scores_df["type"] = [r["type"] for r in results]

metric_cols = ["faithfulness", "answer_relevancy", "context_precision", "context_recall"]

print("=== Per-Question Scores ===")
display_df = scores_df[["question", "type"] + metric_cols].copy()
display_df["question"] = display_df["question"].str[:60] + "..."
display(display_df)

print("\n=== Aggregate Scores ===")
print(scores_df[metric_cols].mean().to_frame("mean").T.to_string(float_format="{:.3f}".format))

print("\n=== Scores by Question Type ===")
print(scores_df.groupby("type")[metric_cols].mean().to_string(float_format="{:.3f}".format))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: aggregate scores
agg = scores_df[metric_cols].mean()
colors = ["#2563eb", "#16a34a", "#d97706", "#dc2626"]
axes[0].bar(agg.index, agg.values, color=colors)
axes[0].set_ylim(0, 1)
axes[0].set_ylabel("Score")
axes[0].set_title("RAGAS Aggregate Scores")
for i, v in enumerate(agg.values):
    axes[0].text(i, v + 0.02, f"{v:.3f}", ha="center", fontweight="bold")

# Grouped bar chart: by question type
type_means = scores_df.groupby("type")[metric_cols].mean()
type_means.plot(kind="bar", ax=axes[1], color=colors, width=0.7)
axes[1].set_ylim(0, 1)
axes[1].set_ylabel("Score")
axes[1].set_title("RAGAS Scores by Question Type")
axes[1].legend(loc="lower right", fontsize=8)
axes[1].tick_params(axis="x", rotation=30)

plt.tight_layout()
plt.show()

---
## 3. Online Evaluation Strategy

Production monitoring uses three complementary signals: **user feedback**, **application telemetry**, and **cost tracking**.

### 3.1 User Feedback (Supabase)

Every assistant message supports thumbs up (+1) / thumbs down (-1) feedback, stored in the `message_feedback` table:

```sql
create table message_feedback (
    message_id uuid primary key references messages(id) on delete cascade,
    signal     smallint not null check (signal in (-1, 1)),
    created_at timestamptz not null default now(),
    updated_at timestamptz not null default now()
);
```

**API endpoints**:
- `PUT /api/messages/{id}/feedback` — submit feedback (`{"signal": 1}` or `{"signal": -1}`)
- `DELETE /api/messages/{id}/feedback` — retract feedback

**Key KPIs**:
- **Satisfaction rate**: `count(signal=1) / count(*)` — target ≥ 80%
- **Feedback coverage**: `count(feedback) / count(assistant_messages)` — measures engagement
- **Dislike clustering**: group dislikes by question type or section to identify weak spots

In [ ]:
# Query feedback data via the API
# Fetch all threads and their messages to compute feedback metrics

threads_resp = httpx.get(f"{API_BASE}/api/threads")
threads_resp.raise_for_status()
threads = threads_resp.json()

all_feedback = []

for thread in threads:
    msgs_resp = httpx.get(f"{API_BASE}/api/threads/{thread['id']}/messages")
    msgs_resp.raise_for_status()
    messages = msgs_resp.json()

    for msg in messages:
        if msg["role"] == "assistant" and msg.get("feedback") is not None:
            all_feedback.append({
                "message_id": msg["id"],
                "signal": msg["feedback"],
                "message_type": msg.get("message_type"),
                "created_at": msg["created_at"],
            })

if all_feedback:
    fb_df = pd.DataFrame(all_feedback)
    total = len(fb_df)
    likes = (fb_df["signal"] == 1).sum()
    dislikes = (fb_df["signal"] == -1).sum()
    print(f"Total feedback: {total}")
    print(f"  Likes:    {likes} ({likes/total:.0%})")
    print(f"  Dislikes: {dislikes} ({dislikes/total:.0%})")
    print(f"\nBy message type:")
    print(fb_df.groupby("message_type")["signal"].agg(["count", "mean"]).to_string())
else:
    print("No feedback data collected yet.")

### 3.2 Application Telemetry (Azure Application Insights)

The backend integrates Azure Monitor OpenTelemetry for production observability:

```python
# backend/app/main.py
from azure.monitor.opentelemetry import configure_azure_monitor
configure_azure_monitor()  # enabled when APPLICATIONINSIGHTS_CONNECTION_STRING is set
```

**Tracked metrics**:

| Metric | Source | Purpose |
|---|---|---|
| Request latency (p50/p95/p99) | Auto-instrumented HTTP spans | Detect slow queries |
| Error rate | HTTP 4xx/5xx responses | Monitor reliability |
| Dependency latency | OpenAI, Pinecone, Supabase calls | Identify bottleneck services |
| Token usage | OpenAI API response headers | Track consumption |
| Active threads / messages | Custom counters | Usage volume |

**Alerting rules** (recommended):
- P95 latency > 10s → investigate retrieval or LLM latency
- Error rate > 5% over 5 min → page on-call
- Dislike rate > 30% over 1 hour → review recent queries

### 3.3 Cost Monitoring

**Per-query cost model** (estimated):

| Component | Cost per query | Notes |
|---|---|---|
| Query routing (GPT-4o-mini) | ~$0.00002 | ~100 input tokens, ~5 output tokens |
| Embedding (text-embedding-3-large) | ~$0.00013 | ~100 tokens per query |
| Pinecone query | ~$0.000008 | Serverless, per-read-unit |
| Generation (GPT-4o) | ~$0.005–0.02 | Varies with context + response length |
| **Total per query** | **~$0.005–0.02** | Dominated by generation cost |

**Monitoring approach**:
- Track OpenAI API usage via the [Usage Dashboard](https://platform.openai.com/usage)
- Pinecone usage via the [Pinecone Console](https://app.pinecone.io/)
- Set billing alerts at $50, $100, $200 thresholds
- Log token counts per request for fine-grained attribution

### 3.4 Continuous Improvement Loop

```
┌─────────────┐     ┌──────────────────┐     ┌───────────────┐
│  User Query  │────▶│  FinRAG Pipeline  │────▶│   Response    │
└─────────────┘     └──────────────────┘     └───────┬───────┘
                                                     │
                     ┌───────────────────────────────┘
                     ▼
              ┌─────────────┐
              │  Feedback   │  thumbs up/down
              └──────┬──────┘
                     │
         ┌───────────┴───────────┐
         ▼                       ▼
  ┌─────────────┐       ┌──────────────┐
  │  Telemetry  │       │  RAGAS Eval   │  periodic re-runs
  │  Dashboard  │       │  (this nb)    │
  └──────┬──────┘       └──────┬───────┘
         │                     │
         └─────────┬───────────┘
                   ▼
          ┌─────────────────┐
          │  Improvements   │  chunking, prompts,
          │  & Tuning       │  retrieval params
          └─────────────────┘
```

**Actionable triggers**:
1. Low **faithfulness** → LLM hallucinating beyond retrieved context → tighten system prompt constraints
2. Low **context recall** → retrieval missing relevant chunks → increase `retrieval_top_k`, tune chunk size
3. Low **context precision** → irrelevant chunks ranked high → improve embedding model or add re-ranking
4. Low **answer relevancy** → model not addressing the question → refine prompt template
5. High dislike rate on tabular questions → table parsing quality → improve Azure DI table extraction